In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from matplotlib import font_manager, rc
import platform

if platform.system() == 'Windows':
# 윈도우인 경우
    font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)
else:    
# Mac 인 경우
    rc('font', family='AppleGothic')
    
plt.style.use('ggplot')
import datetime
import os
np.random.seed(0)

# 1. trend_data

* 네이버 트렌드 랩에서 수집한 데이터
* 직접 추가한 대,중,소,브랜드 컬럼에 대한 트렌드를 수집함
* https://datalab.naver.com/keyword/trendSearch.naver

In [2]:
# tt라는 리스트에 트렌드 데이터를 추가함
tt = []
for i in os.listdir():
    if i[-3:] == 'csv':
        if 'trend_data_2_train' in i:
            print(i)
            try:
                tt.append(pd.read_csv(i,encoding='cp949'))
            except:
                tt.append(pd.read_csv(i))

trend_data_2_train_min.csv
trend_data_2_train_to0from1000.csv
trend_data_2_train_to1000from2000.csv
trend_data_2_train_to2000from3000.csv
trend_data_2_train_to3000from4000.csv
trend_data_2_train_세홍4000_5000.csv
trend_data_2_train_세홍_2_5000_6000.csv


In [3]:
# train과 test에 트렌드 데이터를 추가하기 위한 전처리
train_trend = pd.read_csv('trend_data_2018_scaled.csv', encoding='cp949')
test_trend = pd.read_csv('trend_data_2019_scaled.csv', encoding='cp949')

train_trend = train_trend.set_index('기간').stack().reset_index().rename(columns={'level_1':'item'})
test_trend =test_trend.set_index('기간').stack().reset_index().rename(columns={'level_1':'item'})

train_trend['기간'] = (pd.to_datetime(train_trend['기간']) + datetime.timedelta(days=365)).apply(lambda x:str(x)[0:10])
test_trend['기간'] = (pd.to_datetime(test_trend['기간']) + datetime.timedelta(days=365)).apply(lambda x:str(x)[0:10])

* train

In [4]:
# train데이터에 item_w_mean,1years_ago_trend 컬럼 추가
df = pd.DataFrame()
for i in tt:
    df = pd.concat([df,i])

train = pd.read_csv(r'new_train.csv',index_col=0,encoding='cp949')

train['sdsd'] = train['방송일시'].apply(lambda x:str(x)[0:10])

train = train.merge(df,left_on=['sdsd','item'],right_on=['날짜','item'],how='left')

train = train.merge(train_trend,left_on=['sdsd','item'],right_on=['기간','item'],how='left')

train = train.drop(['sdsd','날짜','item_mean','item_total_mean','기간'],axis=1).rename(columns={0:'1years_ago_trend'})

train['item_w_mean'] = train['item_w_mean'].fillna(0)
train['1years_ago_trend'] = train['1years_ago_trend'].fillna(0)

* test

In [5]:
# test데이터에 item_w_mean,1years_ago_trend 컬럼 추가

df = pd.read_csv('trend_data_2_test.csv',encoding='cp949')
test = pd.read_csv(r'new_test.csv',index_col=0,encoding='cp949')

test['sdsd'] = test['방송일시'].apply(lambda x:str(x)[0:10])

test = test.merge(df,left_on=['sdsd','item'],right_on=['날짜','item'],how='left')

test = test.merge(test_trend,left_on=['sdsd','item'],right_on=['기간','item'],how='left')

test = test.drop(['sdsd','날짜','item_mean','item_total_mean','기간'],axis=1).rename(columns={0:'1years_ago_trend'})

test['1years_ago_trend'] = test['1years_ago_trend'].fillna(0)

# 2. 전력 데이터

* kpx에서 수집한 전력데이터를 추가함
* smp와 가격결정발전계획용 수요예측데이터를 추가함.

* https://www.kpx.or.kr/www/contents.do?key=223
* https://www.kpx.or.kr/www/contents.do?key=225

In [6]:
elc_train = pd.read_excel(r'bidforecastgen_land_2019.xls',header=3)
elc_test = pd.read_excel(r'bidforecastgen_land_2020.xls',header=3)

* 가격결정발전계획용 수요예측데이터

In [7]:
df = pd.concat([elc_train,elc_test])

elc_df = pd.melt(df,id_vars='구분')

elc_df['variable'] = elc_df['variable'].apply(lambda x:'0'+str(x) if len(str(x))==1 else x)

elc_df['구분'] = elc_df['구분'].astype('str')
elc_df['variable'] = elc_df['variable'].astype('str')

elc_df['구분'] = elc_df['구분'] + elc_df['variable']


# 원본데이터에 00시의 데이터가 24시의 데이터로 표현되어 있음
# 24시를 다음날 00시의 데이터로 바꾸는 함수
def change_0000(x):
    tm=x                                
    if x[-2:]=='24':
        tm = datetime.datetime.strptime(x[:-2],'%Y%m%d')
        tm = tm + datetime.timedelta(days=1)
        tm = tm.strftime('%Y%m%d') +'00'
                                        
    return tm


elc_df['구분'] = elc_df['구분'].apply(lambda x: change_0000(x))

elc_df['구분'] = elc_df['구분'].apply(lambda x: x[0:8] + ' ' +x[8:]+':00:00')

elc_df['구분'] = pd.to_datetime(elc_df['구분'])

elc_df = elc_df.rename(columns={'value':'forecast_elc_gen'}).drop('variable',axis=1)


train['방송일시'] = pd.to_datetime(train['방송일시'])
test['방송일시'] = pd.to_datetime(test['방송일시'])

train = pd.merge(train, elc_df, left_on = '방송일시',right_on = '구분',how='left').drop('구분',axis=1)

test = pd.merge(test, elc_df, left_on = '방송일시',right_on = '구분',how='left').drop('구분',axis=1)

* smp

In [8]:
elc_train = pd.read_excel(r'smp_land_2019.xls',header=3)
elc_test = pd.read_excel(r'smp_land_2020.xls',header=3)

df = pd.concat([elc_train,elc_test])

df = df.drop(['최대','최소','평균'],axis=1)



elc_df = pd.melt(df,id_vars='구분')

elc_df['variable'] = elc_df['variable'].apply(lambda x:'0'+str(x) if len(str(x))==1 else x)

elc_df['구분'] = elc_df['구분'].astype('str')
elc_df['variable'] = elc_df['variable'].astype('str')

elc_df['구분'] = elc_df['구분'] + elc_df['variable']


# 원본데이터에 00시의 데이터가 24시의 데이터로 표현되어 있음
# 24시를 다음날 00시의 데이터로 바꾸는 함수
def change_0000(x):
    tm=x                                
    if x[-2:]=='24':
        tm = datetime.datetime.strptime(x[:-2],'%Y%m%d')
        tm = tm + datetime.timedelta(days=1)
        tm = tm.strftime('%Y%m%d') +'00'
                                        
    return tm


elc_df['구분'] = elc_df['구분'].apply(lambda x: change_0000(x))

elc_df['구분'] = elc_df['구분'].apply(lambda x: x[0:8] + ' ' +x[8:]+':00:00')

elc_df['구분'] = pd.to_datetime(elc_df['구분'])

elc_df = elc_df.rename(columns={'value':'smp'}).drop('variable',axis=1)

train = pd.merge(train, elc_df, left_on = '방송일시',right_on = '구분',how='left').drop('구분',axis=1)

test = pd.merge(test, elc_df, left_on = '방송일시',right_on = '구분',how='left').drop('구분',axis=1)

train['forecast_elc_gen'] = train['forecast_elc_gen'].ffill()

train['smp'] = train['smp'].ffill()

test['forecast_elc_gen'] = test['forecast_elc_gen'].ffill().bfill()

test['smp'] = test['smp'].ffill().bfill()

# 3. 지하철 데이터

* https://data.seoul.go.kr/dataList/OA-12921/F/1/datasetView.do _ train
* https://www.data.go.kr/data/15060424/fileData.do

In [9]:
sub_train = pd.read_csv(r'서울교통공사_1~8호선일별역별시간대별승하차인원_2019년.csv',encoding='cp949',header=1)
sub_test = pd.read_csv(r'서울교통공사_1_8호선일별역별시간대별승하차인원_202001_202008.csv',encoding='cp949',header=0)

sub_train = pd.concat([sub_train,sub_test])

sub_train = sub_train.drop(['역명','역번호','호선'],axis=1)

sub_train = pd.melt(sub_train, id_vars=['날짜','구분'])

sub_train = sub_train.groupby(['날짜','구분','variable']).mean().reset_index()

sub_train = sub_train.drop(sub_train[sub_train['variable'] == '합 계'].index)

sub_train['variable'] = sub_train['variable'].apply(lambda x: '05' if x == '06시 이전' else ('24' if x=='24시 이후' else (x[0:2]) ))

sub_train['날짜'] = sub_train['날짜'] + sub_train['variable']


# 원본데이터에 00시의 데이터가 24시의 데이터로 표현되어 있음
# 24시를 다음날 00시의 데이터로 바꾸는 함수
def change_0000(x):
    tm=x                                
    if x[-2:]=='24':
        tm = datetime.datetime.strptime(x[:-2],'%Y-%m-%d')
        tm = tm + datetime.timedelta(days=1)
        tm = tm.strftime('%Y-%m-%d') +'00'
                                        
    return tm

sub_train['날짜'] = sub_train['날짜'].apply(lambda x:change_0000(x))

sub_train['날짜'] = sub_train['날짜'].apply(lambda x: x[0:10] + ' ' +x[10:]+':00:00')

sub_train['날짜'] = pd.to_datetime(sub_train['날짜'])

sub_train = sub_train.drop('variable',axis=1)
sub_train = sub_train.drop(sub_train[sub_train['구분']=='하차'].index)
sub_train = sub_train.drop('구분',axis=1)

train['방송일시'] = pd.to_datetime(train['방송일시'])
test['방송일시'] = pd.to_datetime(test['방송일시'])

train = pd.merge(train, sub_train, left_on = '방송일시',right_on = '날짜',how='left').drop('날짜',axis=1)

test = pd.merge(test, sub_train, left_on = '방송일시',right_on = '날짜',how='left').drop('날짜',axis=1)

train = train.rename(columns = {'value':'subway'})
test = test.rename(columns = {'value':'subway'})

train['subway'] = train['subway'].ffill()

test['subway'] = test['subway'].ffill().bfill()

# 4. 날씨데이터 전처리

In [10]:
df = pd.concat([train,test])

In [11]:
weather_df = pd.read_csv(r'weather_df_(all).csv',encoding='cp949',index_col=0)

In [12]:
weather_df = weather_df.ffill()

weather_df['일시'] = pd.to_datetime(weather_df['일시'], format='%Y-%m-%d %H:%M', errors='raise')

df = pd.merge(df, weather_df, left_on = "방송일시", right_on = "일시", how = 'left')
df.drop('일시', inplace = True, axis = 1)


# 5.미세먼지 데이터 전처리

* https://www.airkorea.or.kr/web/last_amb_hour_data?pMENU_NO=123

In [13]:
dust_df = pd.read_csv(r'dust_df.csv',encoding='cp949',index_col = 0)

In [14]:
dust_df['측정일시'] = dust_df['측정일시'].apply(lambda x: pd.to_datetime(x))

df = pd.merge(df, dust_df, left_on = '방송일시',right_on = '측정일시',how='left').drop('측정일시',axis=1)

In [15]:
df.iloc[:,40:] = df.iloc[:,40:].ffill()

pm10 = df.iloc[:,40:57].mean(axis=1)
pm25 = df.iloc[:,57:].mean(axis=1)

df = df.drop(df.iloc[:,40:].columns,axis=1)

df['pm10'] = pm10
df['pm25'] = pm25

# 6. 기본데이터 전처리

* holiday #http://marketdata.krx.co.kr/mdi#document=01100305

In [16]:
view = pd.read_excel('viewrating.xlsx',header = 1)
view_raw = view.drop(view.index[-1]).copy()
holiday = pd.read_excel('holiday.xls',header = 0) 

* 6.1 결측값 처리

In [17]:
df['방송일시'] = pd.to_datetime(df['방송일시'])

df['취급액'] = df['취급액'].fillna(0)

df['노출(분)'] = df['노출(분)'].ffill()

* 6.2 날짜 데이터 추가
    *    6.2.1 공휴일

In [18]:
holiday['일자 및 요일'] = holiday['일자 및 요일'].apply(lambda x:pd.to_datetime(x))

holi = []
for i in holiday['일자 및 요일']:
    month = i.month
    day = i.day
    holi.append(str(month)+str(day))
    
df['holi'] = df['방송일시'].apply(lambda x: str(x.month)+str(x.day))

df['공휴일'] = df['holi'].isin(holi)

df.drop('holi',axis=1,inplace=True)

    * 6.2.2 요일

In [19]:
def getDayName(x):
    daystring = ['월','화','수','목','금','토','일']
    return daystring[x.weekday()]

df['요일'] = df.방송일시.apply(getDayName)

    * 6.2.3 월초, 중, 말

In [20]:
df1 = df.copy()
df1['일'] = df['방송일시'].dt.day

def choJungMal(x):
    if 0 < x <= 10:
        return "월초"
    elif 11 <= x <= 20:
        return "월중"
    else:
        return "월말"

df['초중말'] = df1['일'].apply(choJungMal)

    * 6.2.4 월,일,시간

In [21]:
df['month'] = df['방송일시'].apply(lambda x:str(x.month))
df['day'] = df['방송일시'].apply(lambda x:str(x.day))
df['hour'] = df['방송일시'].apply(lambda x:str(x.hour))
df['minute'] = df['방송일시'].apply(lambda x:str(x.minute))

 * train, test 분리

In [22]:
train = df.iloc[:37372,: ]

test = df.iloc[37372:,: ]

# 7. train + 시청률

In [23]:
view = view.drop(view.index[-1])
view = view.drop('2019-01-01 to 2019-12-31',axis=1)

view = view.set_index('시간대')

df_dummie = pd.DataFrame(columns=view.iloc[-120:,:].columns, index=view.iloc[-120:,:].index)

view = pd.concat([df_dummie,view])

view.iloc[:120,1] = view.iloc[-120:,0]

for i in range(len(view.columns)-1):
    view.iloc[:120,i+1] = view.iloc[-120:,i]

view = view.reset_index()

view = view.drop(view.iloc[-120:,:].index)

view = view.transpose()

view.columns = list(view.iloc[0,:].values)


view = view.drop('시간대',axis=0).fillna(0).reset_index()

view= view.melt(id_vars ='index')


view['index'] = view['index'] + ' '+ view['variable']

view = view.drop('variable',axis=1)
view['index'] = view['index'].apply(lambda x:pd.to_datetime(x))

view1 = view.set_index('index')['value']

def f(x):
    time = str(x['방송일시'])
    time2 = str(x['방송일시'] + datetime.timedelta(minutes=x['노출(분)']-1))
    return view1[time:time2].mean()

train['시청률'] = train.apply(f,axis=1)

test = test.reset_index().drop('index',axis=1)

# 9. data export

In [24]:
train.to_csv('train_after_processing_weatheropt3.csv',encoding='cp949')
test.to_csv('test_after_processing_weatheropt3.csv',encoding='cp949')